# Capstone Project - The Battle of Neighborhoods: Intro and Data

##### **Disclaimer: Informations ans results from this work are only simulated for the capstone project and do not reflect any actual situation in the area**

# Pre-feasibility study: Power plant site selection in the southern of Thailand

## Introduction: Business Problem <a name="introduction"></a>

According to the latest Power Development Plan, the new power plant must be constructed to meet the increasing electricity demand locating in the southern of Thailand. The plan has been concluded in the previous stage of studies that the type of newly built power plant will be combine-cycle power plant using Natural gas and will be located within area of **Songkhla and Phattalung province**.
The operation of fossil-fired power plant cause both environmental and social effects. Therefore it is the first priority in this stage to locate plant with minimum effects towards the nearby social area.

**The aims of this study is to locate the optimal location of power plant that minimize the impact towards nearby locals.**

## Data <a name="data"></a>

This study uses the following data:
* Latitude and Longitude of each sub-district in the area from [[1]]

* Number and types of venue in the area from Foursqure.

[1]:  https://github.com/codesanook/thailand-administrative-division-province-district-subdistrict-sql?fbclid=IwAR02dmKnvJ9cEx1UXHucR60DSP8vbVukXQEdgg7R4nc-iQ5FbhTuyqVfeCg

### 1. Collects the sub-district latitude longitude coordinate

In [1]:
#import libraries
import requests
import pandas as pd
import csv
import urllib.request
from bs4 import BeautifulSoup
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [2]:
s_province = pd.read_csv("Southern of Thailand 2.csv")
s_province[:12]

,TA_ID,TAMBON,AM_ID,AMPHOE,Province,Latitude,Longitude
0,900803,Choeng Sae,9008,Krasae Sin,Songkhla,7.561,100.356
1,901107,Khlong U Ta Phao,9011,Hat Yai,Songkhla,7.045,100.446
2,900703,Takhria,9007,Ranot,Songkhla,7.789,100.269
3,900705,Ban Mai,9007,Ranot,Songkhla,7.780,100.288
4,900106,Ko Yo,9001,Muaeng Songkhla,Songkhla,7.163,100.542
5,900204,Di Luang,9002,Sathing Phra,Songkhla,7.580,100.406
6,900804,Krasae Sin,9008,Krasae Sin,Songkhla,7.609,100.307
7,900203,Sanam Chai,9002,Sathing Phra,Songkhla,7.547,100.417
8,901509,Hua Khao,9015,Singhanakhon,Songkhla,7.214,100.567
9,900209,Wat Chan,9002,Sathing Phra,Songkhla,7.382,100.461


### 2. Plot sub-districts on map

In [3]:
address = 'Songkhla'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Possible sites are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Possible sites are 6.8790221, 100.5498542.


In [4]:
map_th = folium.Map(location = [latitude, longitude], zoom_start = 9)
for lat, lng, tambon in zip(s_province['Latitude'], s_province['Longitude'], s_province['TAMBON']):
    label = '{}'.format(tambon)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_th)  
    
map_th

### 3. Acquire venue in the area via FOURSQUARE

In [5]:

CLIENT_ID = 'NPIEW4LSZMNYKEBYVXEWY11Z5TYMO3YTCCJOV5LL0RQKDTGH' # your Foursquare ID
CLIENT_SECRET = 'D1HCA04DHJNRQ5BQWWDC5S4X5JFXEDDMTWBWRF3Y4GBZCZYV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NPIEW4LSZMNYKEBYVXEWY11Z5TYMO3YTCCJOV5LL0RQKDTGH
CLIENT_SECRET:D1HCA04DHJNRQ5BQWWDC5S4X5JFXEDDMTWBWRF3Y4GBZCZYV


In [6]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            200)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
th_venues = getNearbyVenues(names = s_province['TAMBON'], latitudes = s_province['Latitude'], longitudes = s_province['Longitude'])

Choeng Sae
Khlong U Ta Phao
Takhria
Ban Mai
Ko Yo
Di Luang
Krasae Sin
Sanam Chai
Hua Khao
Wat Chan
Kradang Nga
Bodan
Bodaeng
Khun Tat Wai
Cha Thing Phra
Ban Han
Hua Khao
Krasae Sin
Chalae
Wat Khanun
Pak Trae
Ram Daeng
Bo Yang
Mae Thom
Ra Wa
Krasae Sin
Na Mosri
Botru
Phang Yang
Tham Nop
Chum Phon
Pak Ro
Wat Son
Hat Yai
Ching Kho
Pa Ching
Bang Klam
Chang
Khlong Daen
Muang Ngam
Choeng Sae
Na Mom
Khlong Hae
Nam Khao
Rong
Choeng Sae
Bang Khiat
Ko Yo
Koyai
Tha Hin
Ban Khao
Thung Yai
Sa Kom
Huai Luek
Ra Not
Takhria
Daen Sa Nguan
Khao Rupchang
Tha Kham
Ko Taeo
Tha Pradu
Khuan Ru
Kho Hong
Khlong Rang
Thung Lan
Khok Muang
Thung Khamin
Thung Wang
Sadao
Khu
Khu Khut
Pa Khat
Taling Chan
Plak Nu
Phichit
Sathing Mo
Ban Mai
Tha Bon
Ban Na
Khlong Re
Phang La
Na Thap
Khlong Pia
Khu Ha Tai
Thepha
Cha Nong
Ra Not
Khu Tao
Nam Noi
Pha Wong
Khuan Lang
Pak Bang
Khuan So
Tha Pho
Saphan Mai Kaen
Khae
Ban Not
Tha Mo Sai
Rong
Cha Nae
Than Khiri
Khao Mikiat
Ban Khao
Ratta Phum
Pian
Na Wa
Sa Ba Yoi
Khlong La
Ban Ph